In [1]:
#!/usr/bin/env python3
"""
Test script for understanding gamete production and recombination
This is the most complex part - where crossing over happens during meiosis
"""

import numpy as np
import random
from dataclasses import dataclass
from typing import List, Dict, Optional

@dataclass
class Marker:
    id: str
    physical_position: float
    genetic_position: float

@dataclass
class Chromosome:
    id: int
    physical_length_bp: float
    genetic_length_cM: float
    markers: List[Marker]

class TestIndividual:
    """Simplified Individual class for testing gamete production"""
    
    def __init__(self, id: str, chromosomes: List[Chromosome]):
        self.id = id
        self.chromosomes = chromosomes
        self.maternal_chroms = {}
        self.paternal_chroms = {}
        
    def initialise_ancestral_chromosomes(self, ancestry: int):
        for chrom in self.chromosomes:
            n_markers = len(chrom.markers)
            self.maternal_chroms[chrom.id] = {
                'alleles': [ancestry] * n_markers,
                'positions': [m.physical_position for m in chrom.markers]
            }
            self.paternal_chroms[chrom.id] = {
                'alleles': [ancestry] * n_markers,
                'positions': [m.physical_position for m in chrom.markers]
            }

    def create_hybrid_individual(self):
        """Create a hybrid for testing recombination"""
        for chrom in self.chromosomes:
            # Maternal chromosome: alternating ancestries
            maternal_alleles = []
            paternal_alleles = []
            
            for i in range(len(chrom.markers)):
                # Create a pattern for easy tracking
                maternal_alleles.append(0 if i % 2 == 0 else 2)  # 0,2,0,2,0...
                paternal_alleles.append(2 if i % 2 == 0 else 0)  # 2,0,2,0,2...
            
            self.maternal_chroms[chrom.id] = {
                'alleles': maternal_alleles,
                'positions': [m.physical_position for m in chrom.markers]
            }
            self.paternal_chroms[chrom.id] = {
                'alleles': paternal_alleles,
                'positions': [m.physical_position for m in chrom.markers]
            }

    def produce_gamete_simple(self, chrom: Chromosome, n_crossovers: int = 0) -> List[int]:
        """
        Simplified gamete production for testing
        
        This simulates meiosis where:
        1. We start with maternal chromosome 
        2. After each crossover, we switch to paternal (then back to maternal, etc.)
        3. The result is a recombined chromosome for the gamete
        """
        print(f"  Producing gamete for chromosome {chrom.id} with {n_crossovers} crossovers")
        
        # Get the parent's chromosome copies
        maternal_alleles = self.maternal_chroms[chrom.id]['alleles']
        paternal_alleles = self.paternal_chroms[chrom.id]['alleles']
        
        print(f"    Parent's maternal chromosome: {maternal_alleles}")
        print(f"    Parent's paternal chromosome: {paternal_alleles}")
        
        # Start with maternal chromosome
        current_source = 0  # 0 = maternal, 1 = paternal
        gamete_alleles = []
        
        # Randomly place crossovers
        crossover_positions = []
        if n_crossovers > 0:
            # Place crossovers randomly between markers
            marker_intervals = list(range(1, len(chrom.markers)))  # Between markers
            crossover_positions = sorted(random.sample(marker_intervals, min(n_crossovers, len(marker_intervals))))
            print(f"    Crossovers will occur before markers at positions: {crossover_positions}")
        
        # Build the gamete chromosome
        crossover_idx = 0
        for marker_idx, marker in enumerate(chrom.markers):
            
            # Check if we need to switch source before this marker
            if crossover_idx < len(crossover_positions) and marker_idx >= crossover_positions[crossover_idx]:
                current_source = 1 - current_source  # Switch source
                crossover_idx += 1
                print(f"    CROSSOVER! Now using {'paternal' if current_source == 1 else 'maternal'} source")
            
            # Take allele from current source
            if current_source == 0:
                allele = maternal_alleles[marker_idx]
                source_name = "maternal"
            else:
                allele = paternal_alleles[marker_idx]
                source_name = "paternal"
            
            gamete_alleles.append(allele)
            print(f"    Marker {marker_idx+1}: taking {allele} from {source_name}")
        
        print(f"    Final gamete chromosome: {gamete_alleles}")
        return gamete_alleles

def create_test_setup():
    """Create test chromosomes and individuals"""
    print("=== CREATING TEST SETUP ===\n")
    
    # Create a simple chromosome with 5 markers
    markers = []
    for i in range(5):
        marker = Marker(
            id=f"M{i+1}",
            physical_position=(i + 1) * 1_000_000,
            genetic_position=(i + 1) * 10.0
        )
        markers.append(marker)
    
    chromosome = Chromosome(
        id=1,
        physical_length_bp=10_000_000,
        genetic_length_cM=50.0,
        markers=markers
    )
    
    print("Created chromosome with markers:")
    for marker in markers:
        print(f"  {marker.id}: {marker.physical_position:,} bp ({marker.genetic_position} cM)")
    
    return [chromosome]

def test_no_crossover():
    """Test gamete production with no recombination"""
    print("\n=== TEST 1: NO CROSSOVER ===\n")
    
    chromosomes = create_test_setup()
    
    # Create a hybrid individual
    parent = TestIndividual("Hybrid_Parent", chromosomes)
    parent.create_hybrid_individual()
    
    print("Parent chromosomes:")
    chrom_id = 1
    print(f"  Maternal: {parent.maternal_chroms[chrom_id]['alleles']}")
    print(f"  Paternal: {parent.paternal_chroms[chrom_id]['alleles']}")
    
    print(f"\nProducing gamete with 0 crossovers:")
    gamete = parent.produce_gamete_simple(chromosomes[0], n_crossovers=0)
    
    print(f"\nResult: Gamete should be identical to maternal chromosome")
    print(f"Expected: {parent.maternal_chroms[chrom_id]['alleles']}")
    print(f"Got:      {gamete}")
    print(f"Match: {'✓' if gamete == parent.maternal_chroms[chrom_id]['alleles'] else '✗'}")

def test_one_crossover():
    """Test gamete production with one crossover"""
    print("\n=== TEST 2: ONE CROSSOVER ===\n")
    
    chromosomes = create_test_setup()
    
    # Create a hybrid individual
    parent = TestIndividual("Hybrid_Parent", chromosomes)
    parent.create_hybrid_individual()
    
    print("Parent chromosomes:")
    chrom_id = 1
    print(f"  Maternal: {parent.maternal_chroms[chrom_id]['alleles']}")
    print(f"  Paternal: {parent.paternal_chroms[chrom_id]['alleles']}")
    
    # Set seed for reproducible crossover placement
    random.seed(42)
    
    print(f"\nProducing gamete with 1 crossover:")
    gamete = parent.produce_gamete_simple(chromosomes[0], n_crossovers=1)
    
    print(f"\nAnalyzing result:")
    maternal = parent.maternal_chroms[chrom_id]['alleles']
    paternal = parent.paternal_chroms[chrom_id]['alleles']
    
    print("Marker  Maternal  Paternal  Gamete   Source")
    print("-----   --------  --------  ------   ------")
    for i in range(len(gamete)):
        if gamete[i] == maternal[i]:
            source = "Maternal"
        elif gamete[i] == paternal[i]:
            source = "Paternal"
        else:
            source = "ERROR!"
        print(f"M{i+1}     {maternal[i]:>8}  {paternal[i]:>8}  {gamete[i]:>6}   {source}")

def test_multiple_crossovers():
    """Test gamete production with multiple crossovers"""
    print("\n=== TEST 3: MULTIPLE CROSSOVERS ===\n")
    
    chromosomes = create_test_setup()
    
    # Create a hybrid individual
    parent = TestIndividual("Hybrid_Parent", chromosomes)
    parent.create_hybrid_individual()
    
    print("Parent chromosomes:")
    chrom_id = 1
    print(f"  Maternal: {parent.maternal_chroms[chrom_id]['alleles']}")
    print(f"  Paternal: {parent.paternal_chroms[chrom_id]['alleles']}")
    
    # Test with 2 crossovers
    random.seed(123)
    
    print(f"\nProducing gamete with 2 crossovers:")
    gamete = parent.produce_gamete_simple(chromosomes[0], n_crossovers=2)
    
    print(f"\nAnalyzing result:")
    maternal = parent.maternal_chroms[chrom_id]['alleles']
    paternal = parent.paternal_chroms[chrom_id]['alleles']
    
    print("Marker  Maternal  Paternal  Gamete   Source")
    print("-----   --------  --------  ------   ------")
    for i in range(len(gamete)):
        if gamete[i] == maternal[i]:
            source = "Maternal"
        elif gamete[i] == paternal[i]:
            source = "Paternal"
        else:
            source = "ERROR!"
        print(f"M{i+1}     {maternal[i]:>8}  {paternal[i]:>8}  {gamete[i]:>6}   {source}")

def test_detect_recombination():
    """Test detection of recombination events"""
    print("\n=== TEST 4: DETECTING RECOMBINATION ===\n")
    
    # Create a scenario where we can detect recombination
    # Parent has alternating ancestries, so any crossover will be visible
    
    chromosomes = create_test_setup()
    parent = TestIndividual("Test_Parent", chromosomes)
    
    # Set up parent with clear pattern for easy detection
    chrom_id = 1
    parent.maternal_chroms[chrom_id] = {
        'alleles': [0, 0, 0, 0, 0],  # All Pop A
        'positions': [m.physical_position for m in chromosomes[0].markers]
    }
    parent.paternal_chroms[chrom_id] = {
        'alleles': [2, 2, 2, 2, 2],  # All Pop B
        'positions': [m.physical_position for m in chromosomes[0].markers]
    }
    
    print("Parent setup for recombination detection:")
    print(f"  Maternal (all 0s): {parent.maternal_chroms[chrom_id]['alleles']}")
    print(f"  Paternal (all 2s): {parent.paternal_chroms[chrom_id]['alleles']}")
    
    # Test multiple scenarios
    for n_co in [0, 1, 2]:
        print(f"\nTest with {n_co} crossover(s):")
        random.seed(42 + n_co)  # Different seed for each test
        
        gamete = parent.produce_gamete_simple(chromosomes[0], n_crossovers=n_co)
        
        # Detect switches in gamete
        switches = 0
        for i in range(1, len(gamete)):
            if gamete[i] != gamete[i-1]:
                switches += 1
                print(f"    SWITCH detected between M{i} and M{i+1}: {gamete[i-1]} -> {gamete[i]}")
        
        print(f"    Total switches detected: {switches}")
        print(f"    Gamete: {gamete}")

if __name__ == "__main__":
    # Run all tests
    test_no_crossover()
    test_one_crossover()
    test_multiple_crossovers()
    test_detect_recombination()
    
    print("\n" + "="*60)
    print("=== SUMMARY OF GAMETE PRODUCTION ===")
    print("✓ Gametes start with maternal chromosome")
    print("✓ Each crossover switches to the other parent's chromosome")
    print("✓ No crossover = identical to maternal chromosome")
    print("✓ Crossovers create recombined chromosomes")
    print("✓ Recombination is detectable when alleles switch between markers")
    print("✓ Multiple crossovers create complex recombination patterns")


=== TEST 1: NO CROSSOVER ===

=== CREATING TEST SETUP ===

Created chromosome with markers:
  M1: 1,000,000 bp (10.0 cM)
  M2: 2,000,000 bp (20.0 cM)
  M3: 3,000,000 bp (30.0 cM)
  M4: 4,000,000 bp (40.0 cM)
  M5: 5,000,000 bp (50.0 cM)
Parent chromosomes:
  Maternal: [0, 2, 0, 2, 0]
  Paternal: [2, 0, 2, 0, 2]

Producing gamete with 0 crossovers:
  Producing gamete for chromosome 1 with 0 crossovers
    Parent's maternal chromosome: [0, 2, 0, 2, 0]
    Parent's paternal chromosome: [2, 0, 2, 0, 2]
    Marker 1: taking 0 from maternal
    Marker 2: taking 2 from maternal
    Marker 3: taking 0 from maternal
    Marker 4: taking 2 from maternal
    Marker 5: taking 0 from maternal
    Final gamete chromosome: [0, 2, 0, 2, 0]

Result: Gamete should be identical to maternal chromosome
Expected: [0, 2, 0, 2, 0]
Got:      [0, 2, 0, 2, 0]
Match: ✓

=== TEST 2: ONE CROSSOVER ===

=== CREATING TEST SETUP ===

Created chromosome with markers:
  M1: 1,000,000 bp (10.0 cM)
  M2: 2,000,000 bp (20